In [1]:
#general modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, sys, time, glob
from astropy.time import Time
import copy
import scipy

#nmma modules
nmmapath = os.getenv('NMMA') # path to nmma from environment variable
sys.path.append(nmmapath) 
import nmma
import nmma.em.utils as utils
import nmma.em.io as io
from nmma.em.model import SimpleKilonovaLightCurveModel,GRBLightCurveModel, SVDLightCurveModel, KilonovaGRBLightCurveModel, GenericCombineLightCurveModel
from nmma.em import training, utils, model_parameters
from nmma.em import analysis

No version information file '.version' found


Install wrapt_timeout_decorator if you want timeout simulations.


In [2]:
import afterglowpy as grb

# Generate a light-curve Bulla 2019 model

In [3]:
model_name = "Bu2019lm"
n_coeff = 3
# The array of times we'll use to examine each lightcurve
tini, tmax, dt = 0.1, 5.0, 0.2
tt = np.arange(tini, tmax + dt, dt)  

# filters to be used
filts = ["u","g","r","i","z","y","J","H","K"] 

In [4]:
# paths
print(os.getcwd())
dataDir = f"{nmmapath}/nmma/tests/data/bulla" # Example absolute path: "/Users/fabioragosta/nmma/nmma/tests/data/bulla"
ModelPath = f"{nmmapath}/nmma/svdmodels" # Example absolute path: "/Users/fabioragosta/nmma/svdmodels"
filenames = glob.glob("%s/*.dat" % dataDir)

/home/nvanbemmel/KNTraP/model_fitting/KNTraP-AT2017gfo


In [5]:
# load data
data = io.read_photometry_files(filenames, filters=filts) 
# Load the model data
training_data, parameters = model_parameters.Bu2019lm_sparse(data)

In [6]:
# two different interpolation types are possible "sklearn_gp" or "tensorflow"
interpolation_type = "sklearn_gp"
training_model=training.SVDTrainingModel(
    model_name,
    copy.deepcopy(training_data),
    parameters,
    tt,
    filts,
    svd_path=ModelPath,
    n_coeff=n_coeff,
    interpolation_type=interpolation_type,
    n_epochs=100
)

The grid will be interpolated to sample_time with interp1d
Model exists... will load that model.


In [ ]:
# tmin, tmax, dt = 0.5, 20.0, 0.1
# sample_times = np.arange(tmin, tmax + dt, dt)

# light_curve_model = SVDLightCurveModel(
#         model_name,
#         sample_times,
#         svd_path=ModelPath,
#         interpolation_type=interpolation_type,
#         model_parameters=training_model.model_parameters,
#     )

In [20]:
utils.filts

AttributeError: module 'nmma.em.utils' has no attribute 'filts'

# 2. Plot AT17gfo light-curve

In [9]:
# import AT2017gfo data
data = pd.read_csv(f'{nmmapath}/example_files/lightcurves/AT2017gfo.dat',header=None,delim_whitespace=True)
data.columns = ['time','filter','mag','magerr']

In [10]:
# convert ISOT time formats into MJD for plotting
from astropy.time import Time
times = data['time'].values.tolist()
t = Time(times,format='isot')
data['MJD'] = t.mjd

In [11]:
data

,time,filter,mag,magerr,MJD
0,2017-08-18T00:00:00.000,ps1::g,17.41,0.02,57983.000
1,2017-08-18T00:00:00.000,ps1::r,17.56,0.04,57983.000
2,2017-08-18T00:00:00.000,ps1::i,17.48,0.03,57983.000
3,2017-08-18T00:00:00.000,ps1::z,17.59,0.03,57983.000
4,2017-08-18T00:00:00.000,ps1::y,17.46,0.01,57983.000
...,...,...,...,...,...
136,2017-09-01T23:29:45.600,2massks,20.17,0.08,57997.979
137,2017-09-03T23:31:12.000,2massks,20.77,0.13,57999.980
138,2017-09-04T23:11:02.400,2massks,20.76,0.35,58000.966
139,2017-09-07T23:15:21.600,2massks,21.46,0.08,58003.969


In [16]:
# colors
data['filter'].unique()
colors = {'sdssu':'purple','ps1::g':'blue','ps1::r':'forestgreen','ps1::i':'gold','ps1::z':'orange','ps1::y':'red','2massj':'lightcoral','2massh':'firebrick','2massks':'darkred'}

In [17]:
# plot light curve
fig, ax = plt.subplots(1,1,figsize=(10,8))
for flt in data['filter'].unique():
    sel_flt = data[data['filter'] == flt]
    ax.errorbar(sel_flt['MJD'],sel_flt['mag'],yerr=sel_flt['magerr'],label=flt, color=colors[flt],fmt='o')
ax.set_ylabel(r'$mag_{AB}$', fontsize=18)
ax.set_xlabel(r'$time~[d]$', fontsize=18)
ax.set_ylim([25,17])
ax.legend(fontsize=18)
plt.show()

/tmp/ipykernel_4637/246691255.py:10: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


# 3. Fit Bulla 2019 model to AT17gfo

In [ ]:
# not sure how to do model fitting in a jupyter notebook. 
# have only been able to do this using the command line:
# light_curve_analysis --model Bu2019lm --svd-path nmma/svdmodels/ --interpolation_type tensorflow --outdir outdir_at2017gfo_d --label injection --prior nmma/priors/Bu2019lm.at2017gfo.prior --tmin 0.1 --tmax 20 --dt 0.5 --error-budget 1 --nlive 1024 --Ebv-max 0 --trigger-time 57982.5285236896 --data nmma/example_files/lightcurves/GW170817.dat --plot

In [ ]:
nmmapath

In [ ]:
# I think it doesn't like the hyphens 
_ = analysis(model='Bu2019lm', svd-path==f'{nmmapath}/nmma/svdmodels', 
             interpolation_type='tensorflow', outdir='./outdir', label='injection',
             prior=f'{nmma}/priors/Bu2019lm.at2017gfo.prior',
             tmin=0.1, tmax=20, dt=0.5, error-budget=1, nlive=1024, Ebv-max=0.0,
             trigger-time=57982.5285236896, data=f'{nmma}/nmma/example_files/lightcurves/GW170817.dat',
             plot=True)

# 4. Using Bulla obtained parameters generate AT17gfo-like light-curves at different redshifts and samples